In [12]:
# By convention, import all modules at the top of the notebook so they don't get in the way later
#!/usr/bin/env python3

from sys import argv
from math import sqrt
 
"""Output primes smaller than an integer
Given a single integer argument n, return a list containing all primes
numbers strictly less than n.
"""
 
def main(argv):
    print(eratosthenes(argv[1]))
    return 0
 
def gen_eratosthenes():
    """Generator prime numbers"""
    '''
    usage:
    p=gen_eratosthenes()
    next(p)
    each time run next(p), it produces a new prime number, start from 2, without end.
    '''
    p = 1
    while True:
        isprime = True
        p += 1
        for x in range(2,int(sqrt(p))+1):
            if p%x == 0:
                isprime = False
                break
        if isprime:
            yield p
 
 
def gen_ints():
    """Generator for positive integers strictly greater than 1."""
    n = 2   #Generator starts at 2, the smallest prime number
    while True:
        yield n
        n += 1
 
def eratosthenes(number):
    """Enter a positive integer greater than 2; return the list 
    of prime numbers strictly less than that integer."""
    try:
        number = int(number)
    except TypeError:
        return 'Argument must not be a list'
    except ValueError:
        return 'Argument must be an integer, not "%s"' % number
 
    if number < 2:
        return 'Argument must be an integer greater than 2, not %d' % number
    else:
        #generate list of integers 2 through n-1
        g = gen_ints()
        primes = [next(g) for _ in range (number-2)]
 
        stopPoint = sqrt(number)
        primePosition = 0
        testPrime = primes[primePosition]
        while testPrime < stopPoint:
            #iterate backwards through list and remove multiples
            for index in range(len(primes)-1,primePosition,-1):
                if primes[index]%testPrime == 0:
                    del primes[index]
            primePosition += 1
            testPrime = primes[primePosition]
        return primes
 
def test_primes():
    """Check that the primes less than 100 are correct """
    number = 100
    computed = eratosthenes(number)
    expected = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,
                53,59,61,67,71,73,79,83,89,97]
    success = computed == expected
    message = 'Computed %s, expected %s' % (computed, expected)
    assert success, message
 
def test_primes_negNum():
    """eatosthenes(n) input is a negative number """
    number = -10
    computed = eratosthenes(number)
    expected = 'Argument must be an integer greater than 2, not -10'
    success = computed == expected
    message = 'Computed %s, expected %s' % (computed, expected)
    assert success, message
 
def test_primes_char():
    """eatosthenes(n) input is a char"""
    number = 'a'
    computed = eratosthenes(number)
    expected = 'Argument must be an integer, not "a"'
    success = computed == expected
    message = 'Computed %s, expected %s' % (computed, expected)
    assert success, message
 
if __name__ == "__main__":
    try:
        main(argv)
    except IndexError:
        #Accurate feedback for n==2 since the IndexError gets caught first
        if len(argv) < 2:
            print('Missing command-line argument.')
        else:
            print('No prime number smaller than 2.')
        exit(1)
    # test functions
    test_primes()
    test_primes_negNum()
    test_primes_char()


Argument must be an integer, not "--matplotlib=inline"


<!--this prints a function in-line with the surrounding text-->
<!--to make it appear, the html comment marks must be removed from the following two lines-->
<!--
This is what $f(x)$ looks like for this example:
$$f(x) = \int_x^\infty y^2\, dy$$
-->

<!--this is a markdown comment-->
<!--this is heading one text-->
# CW04 Prime Number Generator
<!--this is using the Latex equation syntax to print our names in italics-->
$Nengyin\,(Helen)\,Zhu,\, Lance\, Clifner,\, Alexis\, Ford$ 

<!--this is normal body text-->
September 20, 2016

This is Classwork Assignment number 04.  In this project, we are officially switching from python 2 to python 3, and all further classwork will be completed in python 3.

For this project, we are creating a python script that computes a list of all the prime numbers between 1 and *N*, where *N* is a number passed on the command line to the prime number python script.

## The Sieve of Eratosthenes
The algorithm we have implemented to calculate the prime number is the <a href="https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes">Sieve of Eratosthenes</a>.  You can follow this link to see a complete description of the algorithm, along with an animation that visually demonstrates the algorithm's steps.

Our hero, Eratosthenes of Cyrene: 
![Alt text](http://www.conservapedia.com/images/b/bc/Eratosthenes.jpg "Eratosthenes of Cyrene")

### Algorithm Design Goals and Considerations
The goal of our algorithm is to implement the Sieve of Eratosthenes in a straightforward, simple, easy to read pair of nested loops.

Our implementaion of the algorithm uses these steps:
1. create a python list of all integers between 2 (the smallest prime number) and *N*
1. calculate the stopping point, which is the ceiling of the square root of *N*
1. start with the first number in the list
1. check to see if we have reached the stopping point, else loop through each successive element remaining in the list until we reach the stopping point.  Call this list element our Current Prime Factor (CPF).
1. with the inner loop, we move backwards from the end of the list until we reach our CPF element.  Call this backwards moving list element our Target
1. check to see if the CPF divides evenly into the Target.  If it does, then remove the Target from the list (since it is an even multiple of the CPF)
1. advance to the next CPF in the list (the next CPF is a prime number)
1. Lather, rinse, and repeat until the stopping point is reached.

Because we have a mathmetician in our group, we know that you can stop your search for primes once you have reached $\sqrt{N}$.

We chose a list as the fundamental data structure because it is one of the simplest Python data types, and is easy to manipulate for adding elements to the list (creating the initial list of integers between 2 and *N*, as well as removing elements from anywhere inside list which are multiples of the primes numbers the algorithm finds).

The algorithm starts by using the CPFs from the beginning of the list because that is the direction from which you get the biggest bang for the buck.  That is, eliminating all the mutliples of 2 from the list cuts the list in half, and then the multiples of 3's cuts the list by another 1/3;  working from the end of the list would have the list impact on the list length, and would also require a priori knowledge of the prime factors.  Thus, each pass through the list requires fewer (or the same, if no Targets were eliminated) computations than the prior pass.

We eliminate the Target elements from the end of the list because when a list member is removed from a list, the indices for all elements deeper in the list then change.  Trying to sequence forward through a list with changing indices requires much more bookkeeping to keep where you are in the list straight than when you move backwards through the list (all the index changes occur behind you deeper in the list while you move to the shallow end of the list and remain unaffected by those index changes).

Next is our cut at a python implementation of Eratosthenes Sieve.

In [18]:
# insert the prime number function here
eratosthenes( 30 )

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

### Generating Prime Numbers
Creating long lists of prime numbers can start taking up a noticeable amounts of computer time.

Using a generator context, where the generator keeps track of the last point of work and just needs to compute the next element in the desired sequence can save computational time over having a function call that generates an entire sequence from the begining and reports either all elements or the final element in the sequence.

For our prime number generator, we use the following algorithm:
1. Assume that the current iteration value is a Prime Number
1. Increment our Prime Target (PT) to the next value
1. Check to see if the PT is evenly divisble by any number (the Check Number CN) from 2 to the square root of the PT plus 1 (again, thanks to our resident mathematician, we don't have to test every number from 2 to the PT)
1. The first instance where PT is evenly divisible by a CN, we know that PT is not prime so we can stop and move to the next integer number in above the current PT
1. Increment the PT by one and keep looping until we find a PT that is not evenly divisible by any CN
1. If we exahust our list of CN's and none of them result in an evenly divisible result, we have discovered a prime number, so we can break from our loop and report the result to the calling function

The next call to our generator will pick up from where it left off.

The main difference between the initial algorithm implemenation in eratosthenes() and the generator is that eratosthenes worked from a known length, pre-initialized list of numbers and cuts the members of the list down until only primes remained.

The generator works by taking the next larger number and checking to see if that number itself is a prime.  If not, it proceeds to the next integer and keeps checking until if finds a prime number.

In [27]:
#  find the first 10 prime numbers
p = gen_eratosthenes()
for index in range(0,10):  
    print( next(p) )

2
3
5
7
11
13
17
19
23
29


### Benchmarking Implementations
Creating long lists of prime numbers can start taking up a noticeable amounts of computer time.  In addition, algorithms can be efficient in certain ranges of parameter settings and inefficient in others.  Benchmarking these two algorithms across a range of values shows that these algorithms are efficient at differing ranges of prime number sequences.

The Sieve of Eratosthenes is more efficient when generating small lists of prime numbers:  under 1000 prime numbers in a sequence.  The Prime Number Generator is more efficient when generating long sequences of prime numbers:  over 10000 prime numbers in a sequence.

So, knowing your intended need for prime number generation might be a determinant in which of the two algorithms to prefer.

<!--
    4 Primes      0.006 msec      0.05 msec
   25 Primes      0.048 msec      0.09 msec
  168 Primes      0.656 msec      1.15 msec
 1229 Primes     12.6   msec     16.7  msec
 9592 Primes    772     msec    292    msec
78498 Primes 82.         sec  7.17      sec
-->

<table>
  <tr align="center">
    <th>Number of Primes</th>
    <th>Eratosthenes</th> 
    <th>Prime Generator</th>
  </tr>
  <tr align="right">
    <td>4</td>
    <td>0.006 msec</td> 
    <td>0.05 msec</td>
  </tr>
  <tr align=right>
    <td>25</td>
    <td>0.048 msec</td> 
    <td>0.09 msec</td>
  </tr>
  <tr align="right">
    <td>168</td>
    <td>0.656 msec</td> 
    <td>1.15 msec</td>
  </tr>
  <tr align="right">
    <td>1229</td>
    <td>12.6   msec</td> 
    <td>16.7  msec</td>
  </tr>
  <tr align="right">
    <td>9592</td>
    <td>772     msec</td> 
    <td>292    msec</td>
  </tr>
  <tr align="right">
    <td>78498</td>
    <td>82.         sec</td> 
    <td>7.17      sec</td>
  </tr>
</table>

The primary difference in speed between the two algorithms is that the Sieve must calculate every single prime in the desired range, whereas the generator is only calculating the next value in the sequence.  This means that the Sieve generates a list of all numbers between 1 and *N* and it touches ever single last number in the sequence and does so multiple times for many of those numbers.  In addition, traversing a list is time consuming versus dealing with a scalar.

For the generator, it performs, on average, fewer calculations per number.  For example, for all even numbers, it performs the calculation exactly once: Even_Target_Number / 2 leaves no remainder, done with check go to next Target Number.

That said, at the low end, there is not a significant difference in performance between the two algorithms when it comes to terms of human perception.  That is, a human can't tell the difference between a response time of 0.006 msec and 0.05 msec--only if you run the program several thousand times or more would a human be able to begin to notice a difference in performance.  

For that reason, it would be reasonable to implement the prime number generator function only and be content with the results for most practical applications.

Note also, it would be possible to significantly speed up our Sieve algorithm by generating a list of Targets that ended at $\sqrt{N} + 1$ instead of going all the way to *N*.  Since there are no primes to be found beyond *N*, there is no need to check past *N*.  We leave this modification as an exercise for the reader.


In [61]:
%%timeit
primes = eratosthenes( 10000 )
#print( len( primes ) )
# 10 ==> 4 primes
# 100 ==> 25 primes
# 1000 ==> 168 primes
# 10000 ==> 1229 primes
# 100000 ==> 9592 primes
# 1000000 ==> 78498 primes

1 loop, best of 3: 1min 22s per loop


In [60]:
%%timeit
p = gen_eratosthenes()
for index in range(0,1229): next(p)
#print( next(p) )
#%time

1 loop, best of 3: 7.17 s per loop
